<a href="https://colab.research.google.com/github/skarot/AI_Recruiter/blob/main/Speech_Emotion_Recognition_with_librosa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
pip install pydub

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
Root = "/content/drive/MyDrive/behavioral_analytics/audio/speech-emotion-recognition-ravdess-data"
os.chdir(Root)

In [4]:
ls

Actor_01/  Actor_05/  Actor_09/  Actor_13/  Actor_17/  Actor_21/
Actor_02/  Actor_06/  Actor_10/  Actor_14/  Actor_18/  Actor_22/
Actor_03/  Actor_07/  Actor_11/  Actor_15/  Actor_19/  Actor_23/
Actor_04/  Actor_08/  Actor_12/  Actor_16/  Actor_20/  Actor_24/


In [5]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [26]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel,RMSE,ZCR):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
        if RMSE:
          RMSE = np.mean(librosa.feature.rms(y=X).T,axis=0)
          result = np.hstack((result, RMSE))
        if ZCR:
          ZCR = np.mean(librosa.feature.zero_crossing_rate(y=X).T,axis = 0)
          result = np.hstack((result,ZCR))


    return result

In [27]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [28]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob(Root+"/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True, RMSE=True, ZCR=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [29]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [30]:
x_train

array([[-3.91131165e+02,  9.70189571e+00, -2.87988052e+01, ...,
         1.43602106e-03,  3.72976176e-02,  2.09353885e-01],
       [-5.13075134e+02,  1.72460461e+01, -1.90956154e+01, ...,
         3.64136795e-04,  1.58599112e-02,  1.31014934e-01],
       [-4.95576782e+02,  1.16502113e+01, -2.18401623e+01, ...,
         1.02561142e-03,  1.84824094e-02,  1.49498806e-01],
       ...,
       [-4.23208344e+02,  2.74825630e+01, -2.51267109e+01, ...,
         3.13917990e-04,  3.41773592e-02,  8.26280382e-02],
       [-7.00923645e+02,  3.44919281e+01,  8.72677803e+00, ...,
         1.82624299e-05,  1.49572117e-03,  1.23014630e-01],
       [-4.83997894e+02,  2.02410831e+01, -1.78483315e+01, ...,
         1.67585837e-04,  1.65915862e-02,  9.17619978e-02]])

In [31]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [32]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 182


In [33]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [34]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [35]:
#Predict for the test set
y_pred=model.predict(x_test)

In [36]:
y_pred

array(['calm', 'disgust', 'fearful', 'fearful', 'disgust', 'happy',
       'disgust', 'fearful', 'fearful', 'disgust', 'fearful', 'fearful',
       'calm', 'calm', 'happy', 'disgust', 'calm', 'disgust', 'happy',
       'fearful', 'calm', 'calm', 'fearful', 'happy', 'calm', 'happy',
       'disgust', 'disgust', 'happy', 'disgust', 'calm', 'calm', 'calm',
       'fearful', 'fearful', 'fearful', 'fearful', 'fearful', 'calm',
       'disgust', 'happy', 'fearful', 'fearful', 'calm', 'fearful',
       'happy', 'calm', 'fearful', 'calm', 'disgust', 'fearful',
       'fearful', 'fearful', 'disgust', 'happy', 'fearful', 'calm',
       'calm', 'happy', 'disgust', 'disgust', 'fearful', 'fearful',
       'happy', 'fearful', 'happy', 'calm', 'fearful', 'disgust', 'calm',
       'disgust', 'happy', 'calm', 'fearful', 'happy', 'disgust',
       'fearful', 'disgust', 'happy', 'fearful', 'disgust', 'disgust',
       'happy', 'fearful', 'disgust', 'happy', 'calm', 'calm', 'disgust',
       'fearful', 'h

In [37]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 63.54%


In [38]:
from sklearn.metrics import accuracy_score, f1_score

In [39]:
f1_score(y_test, y_pred,average=None)

array([0.68686869, 0.57142857, 0.68686869, 0.58947368])

In [40]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,calm,calm
1,calm,disgust
2,calm,fearful
3,fearful,fearful
4,disgust,disgust
5,happy,happy
6,calm,disgust
7,fearful,fearful
8,fearful,fearful
9,happy,disgust


In [53]:
import pickle
# Writing different model files to file
model_path = "/content/sample_data/models/TonePredictionModel.sav"
with open( model_path, 'wb') as f:
    pickle.dump(model,f)

In [55]:
filename = '/content/sample_data/models/TonePredictionModel.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature(Root+"/Actor_01/03-01-01-01-01-01-01.wav", mfcc=True, chroma=True, mel=True,RMSE=True, ZCR=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['fearful'], dtype='<U7')

In [46]:
feature

array([[-6.93497009e+02,  5.00643921e+01,  5.71450531e-01,
         1.43299656e+01,  3.33637071e+00, -2.54071975e+00,
        -4.05790949e+00, -1.07119999e+01, -7.29413939e+00,
         1.74018872e+00, -4.19064236e+00,  1.95466173e+00,
        -5.24789381e+00,  2.78142977e+00, -3.16756773e+00,
        -3.40008307e+00, -2.37803197e+00, -5.68717539e-01,
        -6.47753334e+00, -1.24320555e+00, -2.80542517e+00,
        -5.43635798e+00, -4.46875393e-01, -3.63516593e+00,
        -2.98372602e+00, -5.63902557e-01, -1.65101993e+00,
        -5.55944741e-01, -3.41018438e+00, -2.24465466e+00,
        -3.13058877e+00, -2.70089960e+00, -1.88821292e+00,
        -5.54154456e-01, -3.96459913e+00, -2.13485193e+00,
        -3.94577074e+00, -1.62457860e+00, -2.03990698e+00,
        -3.62910867e+00,  6.33734703e-01,  6.48760676e-01,
         6.21744275e-01,  6.34553194e-01,  6.60345614e-01,
         6.60463989e-01,  7.00949013e-01,  7.31556356e-01,
         7.46921539e-01,  7.23420441e-01,  7.16119647e-0

In [47]:
from moviepy.editor import *

# Input video file path
video_file_path = '/content/Senior Data Analyst _ Screening Round (2023-06-14 14_09 GMT+5_30).mp4'

# Output audio file path
output_audio_path = 'output_audio.wav'

# Load the video clip
video_clip = VideoFileClip(video_file_path)

# Extract the audio from the video clip
audio_clip = video_clip.audio

# Save the audio as a new file
audio_clip.write_audiofile(output_audio_path)

# Close the video and audio clips
video_clip.close()
audio_clip.close()


MoviePy - Writing audio in output_audio.wav


MoviePy - Done.


In [52]:
from pydub import AudioSegment

# Input audio file path
audio_file_path = output_audio_path

# Load the audio file
audio = AudioSegment.from_wav(audio_file_path)

# Duration of each segment in milliseconds
segment_duration = 5000  # 5 seconds

# Split the audio into segments
segments = [audio[i:i + segment_duration] for i in range(0, len(audio), segment_duration)]

# Output directory to save the segments
output_directory = '/content/sample_data/output_segments/'

# Save each segment as a separate audio file
for i, segment in enumerate(segments):
    segment.export(f"{output_directory}segment_{i+1}.wav", format="wav")


In [56]:
filepath = '/content/sample_data/models/TonePredictionModel.sav'
loaded_model = pickle.load(open(filepath, 'rb')) # loading the model file from the storage

feature=extract_feature('/content/sample_data/output_segments/segment_1.wav', mfcc=True, chroma=True, mel=True,RMSE=True, ZCR=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

  warnings.warn(



ValueError: ignored